In [1]:
import requests
import pandas as pd

In [3]:
res = requests.get('https://election.prothomalo.com/election-result-ajax')
if res.status_code == 200:
    data = res.json()
    seats = data['resultData']['win_nearest_candidates']
    store = []
    for seat_no, seat in seats.items():
        if 'nearest' in seat:
            nearest = seat['nearest']
            store.append((
                nearest['area_name'],
                nearest['area_no'],
                nearest['candidates_name'],
                nearest['party'],
                nearest['vote_cast'],
                nearest['jote']
            ))
        if 'win' in seat:
            win = seat['win']
            store.append((
                win['area_name'],
                win['area_no'],
                win['candidates_name'],
                win['party'],
                win['vote_cast'],
                win['jote']
            ))

In [7]:
df = pd.DataFrame(store, columns=['seat_bn', 'seat_no', 'name', 'party_bn', 'vote', 'jote'])
df.head()

,seat_bn,seat_no,name,party_bn,vote,jote
0,ঝিনাইদহ-৩,83,মোঃ মতিয়ার রহমান,bjai,175857,bjai_jote
1,যশোর-৪,88,মতিয়ার রহমান ফারাজী,bnp,131917,bnp_jote
2,যশোর-৪,88,মোঃ গোলাম রছুল,bjai,176912,bjai_jote
3,বরিশাল-৬,124,মোঃ মাহমুদুন্নবী,bjai,55988,bjai_jote
4,বরিশাল-৬,124,আবুল হোসেন খান,bnp,82217,bnp_jote


In [8]:
df.party_bn.unique()

array(['bjai', 'bnp', 'iab', 'abp', 'bkm', 'sontontro', 'ncp', 'juib',
       'bifront', 'gop', 'bijp', 'bbwp', 'bdp', 'km', 'ldp', 'gonoa'],
      dtype=object)

In [9]:
party_mapping = {
    'bkm': 'Bangladesh Khelafat Mojlish',
    'bnp': 'BNP',
    'gonoa': 'Ganosamhati Andolon (Saki)',
    'iab': 'Islami Andolan Bangladesh',
    'juib': 'Jamiat Ulema-e-Islam Bangladesh',
    'km': 'Khelafat Mojlish',
    'ncp': 'NCP',
    'sontontro': 'Independent',
    'bjai': 'Jamaat',
    'abp': 'AB Party',
    'bifront': 'Bangladesh Islami Front',
    'gop': 'GOP (Nuru)',
    'bijp': 'BJP',
    'bbwp': 'Revolutionary Workers Party of Bangladesh',
    'bdp': 'Bangladesh Development Party',
    'ldp':'LDP',
}

def map_party(x):
    if x in party_mapping.keys():
        return party_mapping[x]
    return x
df['party'] = df.party_bn.apply(map_party)
df.head()

,seat_bn,seat_no,name,party_bn,vote,jote,party
0,ঝিনাইদহ-৩,83,মোঃ মতিয়ার রহমান,bjai,175857,bjai_jote,Jamaat
1,যশোর-৪,88,মতিয়ার রহমান ফারাজী,bnp,131917,bnp_jote,BNP
2,যশোর-৪,88,মোঃ গোলাম রছুল,bjai,176912,bjai_jote,Jamaat
3,বরিশাল-৬,124,মোঃ মাহমুদুন্নবী,bjai,55988,bjai_jote,Jamaat
4,বরিশাল-৬,124,আবুল হোসেন খান,bnp,82217,bnp_jote,BNP


In [11]:
def map_jote(x):
    if x == 'bjai_jote':
        return 'Jamaat-NCP'
    if x == 'bnp_jote':
        return 'BNP'
    return x
df['alliance'] = df.jote.apply(map_jote)
df.head()

,seat_bn,seat_no,name,party_bn,vote,jote,party,alliance
0,ঝিনাইদহ-৩,83,মোঃ মতিয়ার রহমান,bjai,175857,bjai_jote,Jamaat,Jamaat-NCP
1,যশোর-৪,88,মতিয়ার রহমান ফারাজী,bnp,131917,bnp_jote,BNP,BNP
2,যশোর-৪,88,মোঃ গোলাম রছুল,bjai,176912,bjai_jote,Jamaat,Jamaat-NCP
3,বরিশাল-৬,124,মোঃ মাহমুদুন্নবী,bjai,55988,bjai_jote,Jamaat,Jamaat-NCP
4,বরিশাল-৬,124,আবুল হোসেন খান,bnp,82217,bnp_jote,BNP,BNP


In [12]:
dfs = pd.read_csv('data/seat_voters_2026.csv')
dfs.head()

,seat_no,seat,district,division,region,voters,male_voters,female_voters,thirdgender_voters
0,1,Panchagarh-1,Panchagarh,Rangpur,Urban,463700.0,231976.0,231723.0,1.0
1,2,Panchagarh-2,Panchagarh,Rangpur,Rural,417529.0,209708.0,207818.0,3.0
2,3,Thakurgaon-1,Thakurgaon,Rangpur,Urban,511629.0,255053.0,256572.0,4.0
3,4,Thakurgaon-2,Thakurgaon,Rangpur,Rural,340049.0,175232.0,164816.0,1.0
4,5,Thakurgaon-3,Thakurgaon,Rangpur,Rural,364129.0,184701.0,179426.0,2.0


In [14]:
dfm = pd.merge(df, dfs, how='left', on='seat_no')
dfm.head()

,seat_bn,seat_no,name,party_bn,vote,jote,party,alliance,seat,district,division,region,voters,male_voters,female_voters,thirdgender_voters
0,ঝিনাইদহ-৩,83,মোঃ মতিয়ার রহমান,bjai,175857,bjai_jote,Jamaat,Jamaat-NCP,Jhenaidah-3,Jhenaidah,Khulna,Rural,431015.0,217470.0,213542.0,3.0
1,যশোর-৪,88,মতিয়ার রহমান ফারাজী,bnp,131917,bnp_jote,BNP,BNP,Jessore-4,Jessore,Khulna,Rural,453634.0,224371.0,229259.0,4.0
2,যশোর-৪,88,মোঃ গোলাম রছুল,bjai,176912,bjai_jote,Jamaat,Jamaat-NCP,Jessore-4,Jessore,Khulna,Rural,453634.0,224371.0,229259.0,4.0
3,বরিশাল-৬,124,মোঃ মাহমুদুন্নবী,bjai,55988,bjai_jote,Jamaat,Jamaat-NCP,Barisal-6,Barisal,Barisal,Rural,315597.0,160055.0,155538.0,4.0
4,বরিশাল-৬,124,আবুল হোসেন খান,bnp,82217,bnp_jote,BNP,BNP,Barisal-6,Barisal,Barisal,Rural,315597.0,160055.0,155538.0,4.0


In [16]:
dfm = dfm.drop(columns=['seat_bn', 'party_bn', 'jote'])
dfm.head()

,seat_no,name,vote,party,alliance,seat,district,division,region,voters,male_voters,female_voters,thirdgender_voters
0,83,মোঃ মতিয়ার রহমান,175857,Jamaat,Jamaat-NCP,Jhenaidah-3,Jhenaidah,Khulna,Rural,431015.0,217470.0,213542.0,3.0
1,88,মতিয়ার রহমান ফারাজী,131917,BNP,BNP,Jessore-4,Jessore,Khulna,Rural,453634.0,224371.0,229259.0,4.0
2,88,মোঃ গোলাম রছুল,176912,Jamaat,Jamaat-NCP,Jessore-4,Jessore,Khulna,Rural,453634.0,224371.0,229259.0,4.0
3,124,মোঃ মাহমুদুন্নবী,55988,Jamaat,Jamaat-NCP,Barisal-6,Barisal,Barisal,Rural,315597.0,160055.0,155538.0,4.0
4,124,আবুল হোসেন খান,82217,BNP,BNP,Barisal-6,Barisal,Barisal,Rural,315597.0,160055.0,155538.0,4.0


In [17]:
dfm['time'] = pd.Timestamp.utcnow()

In [18]:
dfm.to_csv('data/seat_votes_pa.csv', index=False)